In [22]:
import requests
from bs4 import BeautifulSoup
import re
from html import unescape
import pandas as pd
from datetime import datetime, timedelta

In [23]:
# Fetch the web page
web_page_url = 'https://knkf-sectiepowerliften.nl/kalender/'
response = requests.get(web_page_url)
html = response.text

In [24]:

# Parse the HTML
soup = BeautifulSoup(html, 'html.parser')

# Find elements based on class name "post"
post_elements = soup.find_all('section', class_='post')

In [25]:
# Function to recursively get the entire HTML structure of an element
def get_full_structure(element):
    return "".join([str(tag) for tag in element.contents])

# Print the entire structure of each post element
for index, post in enumerate(post_elements, start=1):
    print(f"=== Post {index} ===")
    print(get_full_structure(post))
    print("=================")


=== Post 1 ===

<div class="row">
<div class="col-md-2">
<div class="image">
<a href="https://knkf-sectiepowerliften.nl/kalender/2023/sbd-cup/">
<img alt="" class="img-responsive" src="https://knkf-sectiepowerliften.nl/img/placeholder.png"/>
</a>
</div>
</div>
<div class="col-md-10">
<h2><a href="https://knkf-sectiepowerliften.nl/kalender/2023/sbd-cup/">SBD Cup</a></h2>
<div class="clearfix">
<p>
                                        9 - 10 sep. 2023
                                           • Nijmegen
                                        </p>
</div>
<p class="intro">Organisatie: NKV Atlas</p>
</div>
</div>

=== Post 2 ===

<div class="row">
<div class="col-md-2">
<div class="image">
<a href="https://knkf-sectiepowerliften.nl/kalender/2023/open-nk-bankdrukken/">
<img alt="" class="img-responsive" src="https://knkf-sectiepowerliften.nl/img/placeholder.png"/>
</a>
</div>
</div>
<div class="col-md-10">
<h2><a href="https://knkf-sectiepowerliften.nl/kalender/2023/open-nk-bankdrukken/

In [26]:
# Define the mapping of abbreviated months to full months
month_mapping = {
    "jan": "January",
    "feb": "February",
    "mar": "March",
    "apr": "April",
    "mei": "May",
    "jun": "June",
    "jul": "July",
    "aug": "August",
    "sep": "September",
    "okt": "October",
    "nov": "November",
    "dec": "December"
}



def ordinal_suffix(day):
    return str(day)+("th" if 4<=day%100<=20 else {1:"st",2:"nd",3:"rd"}.get(day%10, "th"))

def dtStylish(dt, compact=False):

    return dt.strftime(f"%a the {{th}}{' of %B %Y' if not compact else ''} ").replace("{th}", ordinal_suffix(dt.day))


regex_pattern_two_days = r'(\d{1,2})\s*-\s*(\d{1,2})\s*([a-zA-Z]+)'
# This function expects a string of the format "## - ## AAA." where # represents a number from 1-31 and A represents any letter
# I use regex to convert the string to a datetime object
def get_signup_date(date_string):
    try:
        # The regular expression splits the string into the starting date, ending date and abbreviated month
        start_day, end_day, month_abbrev = re.match(regex_pattern_two_days, date_string).groups() 
    except AttributeError:
        # In case the event is only one day, there is only one date given, so we can just split
        #print(f"Found single day: {date_string.split(' ')}")
        start_day, month_abbrev = date_string.split(" ")
        end_day = start_day

    # Get the current year since it's not included in the string
    current_year = datetime.now().year

    # Create datetime objects
    start_date = datetime(current_year, list(month_mapping.keys()).index(month_abbrev) + 1, int(start_day))
    end_date = datetime(current_year, list(month_mapping.keys()).index(month_abbrev) + 1, int(end_day))
    
    # Now if we find that the upcoming event is in the past, that means that the event will actually be next year instead of this year. 
    # So we take the same month and day but just increment the year by 1.
    if datetime.now() > start_date:
        start_date = datetime(current_year+1, list(month_mapping.keys()).index(month_abbrev) + 1, int(start_day))
        end_date = datetime(current_year+1, list(month_mapping.keys()).index(month_abbrev) + 1, int(end_day))
        
    # The KNKF has put on their website that they will open the competition for signup 60 days before the competition. 
    signup_date = start_date-timedelta(days=60)
    return signup_date, start_date, end_date


In [27]:
# Define a class to hold event information
class Event:
    def __init__(self, name, signup_date, start_date, end_date, location, organisation, link, notes=None):
        self.name = name
        self.signup_date = signup_date
        self.start_date = start_date
        self.end_date = end_date
        self.location = location
        self.organisation = organisation
        self.link = link
        self.notes = notes
        
    def __str__(self):
        return (
            f"Event Name: {self.name}\n"
            f"Early Signup Deadline: {dtStylish(self.signup_date)}\n"
            f"Date of Event: {dtStylish(self.start_date, True)}to {dtStylish(self.end_date)}\n"
            f"Location: {self.location}\n"
            f"Organisation: {self.organisation}\n"
            f"Link: {self.link}\n"
            f"Notes: {self.notes}\n"
        )
    
    def __eq__(self, other):
        if isinstance(other, Event):
            return (
                self.name == other.name and
                self.signup_date == other.signup_date and
                self.start_date == other.start_date and
                self.end_date == other.end_date and
                self.location == other.location and
                self.organisation == other.organisation and
                self.link == other.link and
                self.notes == other.notes
            )
        return False
            
    def event_to_dataframe(self):
        data = {
            "Name": [self.name],
            "Signup Date": [self.signup_date],
            "Start Date": [self.start_date],
            "End Date": [self.end_date],
            "Location": [self.location],
            "Organisation": [self.organisation],
            "Link": [self.link],
            "Notes": [self.notes]
        }
        df = pd.DataFrame(data)
        return df
    
    @classmethod
    def from_dataframe(cls, df):
        return cls(
            name=df["Name"].iloc[0],
            signup_date=df["Signup Date"].iloc[0],
            start_date=df["Start Date"].iloc[0],
            end_date=df["End Date"].iloc[0],
            location=df["Location"].iloc[0],
            organisation=df["Organisation"].iloc[0],
            link=df["Link"].iloc[0],
            notes=df["Notes"].iloc[0]
        )

In [28]:
# List to store event objects
events = []
# We have two kinds of posts on the page. They vary enough that they should be handled separately.
# The first type is for competitions coming soon; they are already open for signup with the KNKF.
for post in post_elements:
    if not "style" in post.attrs:
        continue    # These posts have a style attribute in the html, while the later competitions do not
    
    name_element = post.find('h2').find('a')
    event_name = name_element.text.strip() if name_element else ""
    if "Masters" in event_name:
        continue    # Since we never have Masters athletes, we don't need to announce these competitions
    
    event_link = name_element['href'] if name_element else ""
    
    date_location_element = post.find('p')
    event_date_location = date_location_element.get_text(strip=True) if date_location_element else ""
    event_date, event_location = event_date_location.split("•") if "•" in event_date_location else ("", "")
    signup_date, start_date, end_date = get_signup_date(event_date)
    # event_date = event_date.split(" ")
    # event_date = month_mapping[event_date[3][:-1]] + " " + "".join(event_date[0:3]) + " " + event_date[4] 
    
    organisation_element = post.find('p', class_='intro')
    event_organisation = organisation_element.text.strip() if organisation_element else "No organisation found"
    # Create Event object and add it to the list
    event = Event(event_name.strip(), signup_date, start_date, end_date, event_location.strip(), event_organisation.strip()[13:], event_link.strip(), "Open for Signup!")
    events.append(event)
    
# Extract information from the competitions coming later. These are defined in the last post
post = post_elements[-1]
date_pattern = re.compile(r'\d{1,2}\s+[a-zA-Z&]{2,}')  # Date pattern in Dutch
date_paragraphs = []

for paragraph in post.find_all('p'):
    #print(paragraph)
    paragraph_text = unescape(paragraph.get_text())  # Decode HTML entities
    #print(paragraph_text)
    if "Wedstrijdinschrijvingen openen 60 dagen" in paragraph_text:
        break
    if date_pattern.search(paragraph_text):
        date_paragraphs.append(paragraph_text.split('\n'))

for paragraph in date_paragraphs:
    event_notes = []
    #print(f"par split returns: {paragraph[0].split(':')}")
    event_date, event_name = paragraph[0].split(':')
    if any([word in event_name.lower() for word in ["masters", "vergadering"]]):
        continue
    event_date = event_date.replace("&", "-")
    if "-" in event_date:
        event_date = event_date.split()
        event_date = " ".join(event_date[0:-1]) + " " + event_date[-1][0:3]
    else:
        event_date = event_date.split(" ")
        event_date = event_date[0] + " " + event_date[1][0:3]
    #print(event_date)
    signup_date, start_date, end_date = get_signup_date(event_date)
    if signup_date < events[-1].signup_date:
        signup_date += timedelta(days=365)
        start_date += timedelta(days=365)
        end_date += timedelta(days=365)
    # event_date = event_date + " " + dtStylish(signup_date)[-4:]
    # event_date = event_date.split(" ")
    # event_date = month_mapping[event_date[3]] + " " + "".join(event_date[0:3]) + " " + event_date[4] 
    
    if '&' in event_name:
        event_name = event_name.split('&')[0]
    location_line = paragraph[1].split('-')
    if len(location_line) > 2:
        event_notes.append(location_line[2].strip().capitalize() +".")
    event_location = location_line[0]
    event_organisation = location_line[1]
    if len(paragraph) > 2:
        event_notes.append(paragraph[2].strip().capitalize())
    event_notes = ' '.join(event_notes)
    event = Event(event_name.strip(), signup_date, start_date, end_date, event_location.strip(), event_organisation.strip(), "No link available yet", event_notes)
    events.append(event)
    

for event in events:
    print(f"{event}")

Event Name: SBD Cup
Early Signup Deadline: Tue the 11th of July 2023 
Date of Event: Sat the 9th to Sun the 10th of September 2023 
Location: Nijmegen
Organisation: NKV Atlas
Link: https://knkf-sectiepowerliften.nl/kalender/2023/sbd-cup/
Notes: Open for Signup!

Event Name: Open NK Bankdrukken & Speijers Sports Cup
Early Signup Deadline: Wed the 2nd of August 2023 
Date of Event: Sun the 1st to Sun the 1st of October 2023 
Location: Beuningen
Organisation: Speijers Sports
Link: https://knkf-sectiepowerliften.nl/kalender/2023/open-nk-bankdrukken/
Notes: Open for Signup!

Event Name: Odin Cup
Early Signup Deadline: Tue the 8th of August 2023 
Date of Event: Sat the 7th to Sun the 8th of October 2023 
Location: Eindhoven
Organisation: ESKV Odin
Link: https://knkf-sectiepowerliften.nl/kalender/2023/odin-cup/
Notes: Open for Signup!

Event Name: Open Bankdruk Cup Classic
Early Signup Deadline: Wed the 6th of September 2023 
Date of Event: Sun the 5th to Sun the 5th of November 2023 
Locatio

In [49]:
from google.oauth2 import credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.auth.transport.requests import Request
import os




def get_credentials(type):
    
    if type.lower() == "drive":
        SCOPES = ["https://www.googleapis.com/auth/drive.file"]  # Use the appropriate scope for the Google Drive API
        TOKEN_FILE = "drive_token.json"
    elif type.lower() == "forms":
        SCOPES = ['https://www.googleapis.com/auth/forms.body', 
                  'https://www.googleapis.com/auth/forms.responses.readonly']
        TOKEN_FILE = "form_token.json"
    elif type.lower() == "scripts":
        SCOPES = ['https://www.googleapis.com/auth/script.projects', 'https://www.googleapis.com/auth/forms']
        TOKEN_FILE = "scripts_token.json"
    else:
        print("Unsupported credential type. Please enter a supported type or implement the missing type.")
        return None
    creds = None
    if os.path.exists(TOKEN_FILE):
        creds = Credentials.from_authorized_user_file(TOKEN_FILE, SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file("client_secrets.json", SCOPES)
            creds = flow.run_local_server(port=0)
        with open(TOKEN_FILE, "w") as token:
            token.write(creds.to_json())
    return creds


In [50]:
# Setup authentication and authorization with the drive and forms APIs
drive_creds = get_credentials("drive")
forms_creds = get_credentials("forms")

DRIVE_DISCOVERY_DOC = "https://www.googleapis.com/discovery/v1/apis/drive/v3/rest"
FORMS_DISCOVERY_DOC = "https://forms.googleapis.com/$discovery/rest?version=v1"

drive_service = build("drive", "v3", credentials=drive_creds, discoveryServiceUrl=DRIVE_DISCOVERY_DOC)
forms_service = build('forms', 'v1', credentials=forms_creds, discoveryServiceUrl=FORMS_DISCOVERY_DOC, static_discovery=False)

# ID of the google drive folder to work in 
folder_id = "12xhQ_a1WQj4sJu7jwHzvdtbrKaPaVDcW"


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=743378426265-9rgg0ob7aasql5fi7llf6dmegk1u6vu4.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A51332%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fforms.body+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fforms.responses.readonly&state=RCTJILk8lvxYUe1Jc1M8upk0D5dBwA&access_type=offline


# Create new form

In [ ]:

# Request body for creating a Google Form file in the specified folder
NEW_FORM = {
    "name": "Competition Signup Form",
    "parents": [folder_id],
    "mimeType": "application/vnd.google-apps.form"
}

with open("form_description.txt", "r", encoding="utf-8") as file:
    new_description = file.read()

update_settings = {
    "requests" : [{
        "updateSettings" : {
            "settings" : {
                
            },
            "updateMask" : ""
        }
    }
    ]
}

update_description = {
    "requests" : [{
        "updateFormInfo" : {
            "info" : {
                "description": new_description
            },
            "updateMask" : "description"
        }
    }]
}

name_question = {
    "requests": [{
        "createItem": {
            "item": {
                "title": "What is your name?",
                "questionItem": {
                    "question": {
                        "required": True,
                        "textQuestion": {}
                    }
                },
            },
            "location": {
                "index": 0
            }
        }
    }]
}

competitions = [{"value": f"{event.name} on {dtStylish(event.start_date, True)}to {dtStylish(event.end_date)}in {event.location}"} for event in events]

competition_question = {
    "requests": [{
        "createItem": {
            "item": {
                "title": "Select all competitions you want to sign up for:",
                "questionItem": {
                    "question": {
                        "required": True,
                        "choiceQuestion": {
                            "type": "CHECKBOX",
                            "options": competitions,
                            "shuffle": False
                        }
                    }
                },
            },
            "location": {
                "index": 1
            }
        }
    }]
}

weightclass_question = {
    "requests": [{
        "createItem": {
            "item": {
                "title": "What weightclass do you want to compete in?",
                "questionItem": {
                    "question": {
                        "required": True,
                        "choiceQuestion": {
                            "type": "DROP_DOWN",
                            "options": [
                                {"value": "F47-"},
                                {"value": "F52-"},
                                {"value": "F57-"},
                                {"value": "F63-"},
                                {"value": "F69-"},
                                {"value": "F76-"},
                                {"value": "F84-"},
                                {"value": "F84+"},
                                {"value": "M59-"},
                                {"value": "M66-"},
                                {"value": "M74-"},
                                {"value": "M83-"},
                                {"value": "M93-"},
                                {"value": "M105-"},
                                {"value": "M120-"},
                                {"value": "M120+"},
                            ],
                            "shuffle": False
                        }
                    }
                },
            },
            "location": {
                "index": 2
            }
        }
    }]
}

squat_question = {
    "requests": [{
        "createItem": {
            "item": {
                "title": "Enter an estimate for your potential squat for this competition. This is used to determine the order of the lifters.",
                "questionItem": {
                    "question": {
                        "required": True,
                        "textQuestion": {}
                    }
                },
            },
            "location": {
                "index": 3
            }
        }
    }]
}

bench_question = {
    "requests": [{
        "createItem": {
            "item": {
                "title": "Enter an estimate for your potential bench press for this competition. This is used to determine the order of the lifters.",
                "questionItem": {
                    "question": {
                        "required": True,
                        "textQuestion": {}
                    }
                },
            },
            "location": {
                "index": 4
            }
        }
    }]
}

deadlift_question = {
    "requests": [{
        "createItem": {
            "item": {
                "title": "Enter an estimate for your potential deadlift for this competition. This is used to determine the order of the lifters.",
                "questionItem": {
                    "question": {
                        "required": True,
                        "textQuestion": {}
                    }
                },
            },
            "location": {
                "index": 5
            }
        }
    }]
}

# Add request to remove the initial question
remove_question = {
    "requests": [{
        "deleteItem": {
            "location": {
                "index": 6
            }
        }
    }]
}
try:
    # Create the Google Form file in the specified folder
    form_file = drive_service.files().create(body=NEW_FORM, media_body=None).execute()
    
    form_id = form_file['id']
    
    new_questions=[update_description, name_question, competition_question, weightclass_question, squat_question, bench_question, deadlift_question, remove_question]
    # Add your code to add questions to the form (similar to your previous code)
    print(f"Form with form ID {form_id} created in folder with ID: {folder_id}")
    
    
    #print(get_result)
    # Adds the question to the form
    for new_question in new_questions:
        question_setting = forms_service.forms().batchUpdate(formId=form_id, body=new_question).execute()

except HttpError as error:
    print(f"An HTTP error occurred: {error}")

# The following request runs a script in google apps scripts API.
# For now, it changes three things: 
# Set RequireLogin to False so that anyone can respond to the form and not just users in the organization.
# Set ShowLinkToRespondAgain to True so that you can easily respond to the form multiple times.
# Link the form response output to an existing sheet in the same folder

script_id = 'AKfycbwxlIO07heL90qgH-ZIWFRjgFrW1Il_QlvUWPvrp8c7EIwAtREAWdCaTzdr2JaIINNilg'
creds = get_credentials("scripts")

service = build('script', 'v1', credentials=creds)

# Create an execution request object.
request = {
        'function': 'updateFormSettings',
        'parameters': [form_id]
    }

try:
    # Make the API request.
    response = service.scripts().run(scriptId=script_id,
                                        body=request).execute()

except HttpError as error:
    # The API encountered a problem before the script started executing.
    print(f"An error occurred: {error}")
    print(error.content)

# Store the new form id into a txt file so we can easily access it later. 
with open("current_form_id.txt", "w") as file:
    file.write(form_id)

# Load and compare events

In [37]:
import pickle
import glob

def save_events_to_file(events_list):
    with open(f"events_{datetime.today().strftime('%Y_%m_%d')}.pkl", "wb") as file:
        pickle.dump(events_list, file)

def load_previous_events_from_file():
    with open(f"{glob.glob('*.pkl')[0]}", "rb") as file:
        events_list = pickle.load(file)
    return events_list

def find_new_events(events):
    loaded_events = load_previous_events_from_file()
    return [event for event in events if event not in loaded_events]

# Save the list to a file
save_events_to_file(events)

# Compare the events in the file to the events pulled from the site
new_events = find_new_events(events)
print(new_events)


[]


# To do:
# Open old form
# Retrieve all responses
# Put responses into a file
# Delete old form
# Generate new form 

In [209]:
from tabulate import tabulate
import numpy as np
with open("current_form_id.txt", 'r') as form_id_file:
    try:
        form_responses = forms_service.forms().responses().list(formId=form_id_file.read()).execute()
    except Exception as e:
        print(e)

form_response = form_responses['responses'][0]

# createTime is when the form response was submitted
createTime = form_response['createTime'][0:-1]
# Now the answers are extracted
__, answer_data = zip(*form_response['answers'].items())
answers = [answer['textAnswers']['answers'] for answer in answer_data]


for i in range(len(answers[1])-1):
    answers[0].append(answers[0][0])
    answers[2].append(answers[2][0])
    answers[3].append(answers[3][0])
    answers[4].append(answers[4][0])
    answers[5].append(answers[5][0])

answers = np.asarray(answers)
# Initialize lists to store data
rows = []

# Loop through the data list
for i in range(np.shape(answers)[1]):
    name = answers[0][i].get('value', None)
    competition = answers[1][i].get('value', None)
    weightclass = answers[2][i].get('value', None)
    squat = answers[3][i].get('value', None)
    bench = answers[4][i].get('value', None)
    deadlift = answers[5][i].get('value', None)
    
    row = {
        'Time of Response': createTime,
        'Name': name,
        'Competition': competition,
        'Weightclass': weightclass,
        'Squat': squat,
        'Bench': bench,
        'Deadlift': deadlift
    }
    rows.append(row)

# Create a DataFrame
df = pd.DataFrame(rows)

print(tabulate(df, headers='keys', tablefmt='psql'))

+----+-------------------------+--------+---------------------------------------------------------------------------------------+---------------+---------+---------+------------+
|    | Time of Response        | Name   | Competition                                                                           | Weightclass   |   Squat |   Bench |   Deadlift |
|----+-------------------------+--------+---------------------------------------------------------------------------------------+---------------+---------+---------+------------|
|  0 | 2023-08-20T11:21:28.689 | Daniel | Open Bankdruk Cup Classic on Sun the 5th to Sun the 5th of November 2023 in Eindhoven | M83-          |       2 |       5 |          1 |
|  1 | 2023-08-20T11:21:28.689 | Daniel | Benelux Cup on Sat the 25th to Sun the 26th of November 2023 in Tilburg               | M83-          |       2 |       5 |          1 |
+----+-------------------------+--------+----------------------------------------------------------------

In [210]:
import gspread
import gspread_dataframe as gd

sheet_id = "1WXn5hOe_UzKoiXAQrdaZ6xm6WVCR0tKImuYYkTsI0wo"

gc = gspread.oauth(
    credentials_filename="client_secrets.json",
    authorized_user_filename='gspread_token.json'
)

def next_available_row(worksheet):
    return len(worksheet.col_values(1))+1

sh = gc.open_by_key(sheet_id)
print(sh.worksheets())
worksheet = sh.worksheets()[0]
print(next_available_row(worksheet))
gd.set_with_dataframe(worksheet=worksheet,dataframe=df,include_index=False,include_column_header=False,row=next_available_row(worksheet),resize=False)
        

[<Worksheet 'Sheet1' id:0>]
5
